<a href="https://colab.research.google.com/github/Zhaxxon/Prob_Stat/blob/master/Course_project/IMDb_analysing_course_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd


**Loading IMDb Dataset**

Each dataset is contained in a gzipped, tab-separated-values (TSV) formatted file in the UTF-8 character set. The first line in each file contains headers that describe what is in each column. A ‘\N’ is used to denote that a particular field is missing or null for that title/name. The available datasets are as follows:

In [2]:
title_basics = pd.read_csv('drive/My Drive/project_data/title.basics.tsv.gz', sep='\t', encoding='utf-8')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


**title.basics.tsv.gz** - Contains the following information for titles:


*   tconst (string) - alphanumeric unique identifier of the title
*   titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
*   primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
*   originalTitle (string) - original title, in the original language
*   isAdult (boolean) - 0: non-adult title; 1: adult title
*   startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year
*   endYear (YYYY) – TV Series end year. ‘\N’ for all other title types
runtimeMinutes – primary runtime of the title, in minutes
*   genres (string array) – includes up to three genres associated with the title

In [3]:
title_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [0]:
title_ratings = pd.read_csv('drive/My Drive/project_data/title.ratings.tsv.gz', sep='\t', encoding='utf-8')

**title.ratings.tsv.gz** – Contains the IMDb rating and votes information for titles:
*   tconst (string) - alphanumeric unique identifier of the title
*   averageRating – weighted average of all the individual user ratings
*   numVotes - number of votes the title has received

In [5]:
title_ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.6,1609
1,tt0000002,6.0,197
2,tt0000003,6.5,1287
3,tt0000004,6.1,121
4,tt0000005,6.1,2052


**We merge our 2 dataFrame "title_basics" and "title_ratings".**

In [0]:
data_merged_total = pd.merge(title_basics, title_ratings, on=['tconst'])

In [7]:
data_merged_total.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",5.6,1609
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",6.0,197
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",6.5,1287
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short",6.1,121
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",6.1,2052


In [8]:
data_merged_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1031808 entries, 0 to 1031807
Data columns (total 11 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   tconst          1031808 non-null  object 
 1   titleType       1031808 non-null  object 
 2   primaryTitle    1031808 non-null  object 
 3   originalTitle   1031808 non-null  object 
 4   isAdult         1031808 non-null  int64  
 5   startYear       1031808 non-null  object 
 6   endYear         1031808 non-null  object 
 7   runtimeMinutes  1031808 non-null  object 
 8   genres          1031806 non-null  object 
 9   averageRating   1031808 non-null  float64
 10  numVotes        1031808 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 94.5+ MB


**Dtype of runtimeMinutes startYear we change to int. Because, further we need this runtimeMinutes when we sort out the movies as per duration. Before proceeding we need to get rid of from the missing values in runtimeMinutes**

In [0]:
data_merged_total = data_merged_total[(((data_merged_total.runtimeMinutes != '\\N') & (data_merged_total.runtimeMinutes != 'Reality-TV')) & (data_merged_total.runtimeMinutes != 'Talk-Show'))]

In [0]:
data_merged_total = data_merged_total[data_merged_total.startYear != '\\N']

In [0]:
data_merged_total['runtimeMinutes'] = data_merged_total['runtimeMinutes'].astype(np.int64)
data_merged_total['startYear'] = data_merged_total['startYear'].astype(np.int64)

In [12]:
data_merged_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 752735 entries, 0 to 1031806
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          752735 non-null  object 
 1   titleType       752735 non-null  object 
 2   primaryTitle    752735 non-null  object 
 3   originalTitle   752735 non-null  object 
 4   isAdult         752735 non-null  int64  
 5   startYear       752735 non-null  int64  
 6   endYear         752735 non-null  object 
 7   runtimeMinutes  752735 non-null  int64  
 8   genres          752735 non-null  object 
 9   averageRating   752735 non-null  float64
 10  numVotes        752735 non-null  int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 68.9+ MB


**We check how many types of videos we have in our dataset.**

In [13]:
title_types = data_merged_total.titleType.unique()
title_types


array(['short', 'movie', 'tvMovie', 'tvSeries', 'tvEpisode', 'tvShort',
       'tvMiniSeries', 'video', 'tvSpecial', 'videoGame'], dtype=object)

**Here we sort out the movies from our dataset**

In [14]:
movies = data_merged_total[data_merged_total.titleType == 'movie']
movies

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance,5.9,152
145,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,20,"Documentary,News,Sport",5.2,343
331,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N,4.4,5
365,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Biography,Crime,Drama",6.1,570
424,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,\N,120,"Adventure,Fantasy",5.0,35
...,...,...,...,...,...,...,...,...,...,...,...
1031755,tt9914642,movie,Albatross,Albatross,0,2017,\N,97,Documentary,8.2,24
1031756,tt9914644,movie,9/11: Escape from the Towers,9/11: Escape from the Towers,0,2018,\N,120,Documentary,8.4,38
1031761,tt9914942,movie,La vida sense la Sara Amat,La vida sense la Sara Amat,0,2019,\N,74,Drama,6.8,78
1031791,tt9916160,movie,Drømmeland,Drømmeland,0,2019,\N,72,Documentary,6.6,36


In [15]:
gen = movies_2020.genres.unique()
gen


NameError: ignored

In [0]:
movies_2020 = movies[(movies.runtimeMinutes > 60) & (movies.startYear == 2020)]
movies_2020

In [0]:
movies_2020.describe()